In [1]:
# Kerakli kutubxonalarni chaqirib olish
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

# Training paramterlari
batch_size = 64  # batch o'lchami 64
device = 'cuda' if cuda.is_available() else 'cpu'  # GPU (cuda)da train(agar mavjud bo'lsa)
print(f'MNIST modelini {device} da train qilish !\n{"=" * 44}')

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader 
train_loader = data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class MNIST_Net(nn.Module):

    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Ma'lumotni reformalash (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)


model = MNIST_Net() #model chaqirib olish
model.to(device)  # cuda ga joylash (agar mavjud bo'lsa)
criterion = nn.CrossEntropyLoss()  # loss (CrossEntropy)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)  # optimizer SGD

# Training uchun funksiya
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Holati: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

# Test uchun funkisya
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # batchdagi loss larni yig'indisini hisoblash
        test_loss += criterion(output, target).item()
        # max qiymatning indeksini olish
        pred = output.data.max(1, keepdim=True)[1]   # torch.max() funkisyasi har ikkala qiymat va indekslarni qaytaradi
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest : Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')


if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training uchun ketgan vaqt: {m:.0f}m {s:.0f}s')
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Test uchun ketgan vaqt: {m:.0f}m {s:.0f}s')

    m, s = divmod(time.time() - since, 60)
    print(f'Umumiy vaqt: {m:.0f}m {s:.0f}s\nModel  {device} qurilmada train qilindi!')

MNIST modelini cpu da train qilish !


100%|███████████████████████████████████████████████████████████████████| 9912422/9912422 [00:07<00:00, 1322105.93it/s]


Extracting ./mnist_data/MNIST\raw\train-images-idx3-ubyte.gz to ./mnist_data/MNIST\raw



100%|███████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 1884236.71it/s]


Extracting ./mnist_data/MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist_data/MNIST\raw



100%|███████████████████████████████████████████████████████████████████| 1648877/1648877 [00:01<00:00, 1189989.44it/s]


Extracting ./mnist_data/MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST\raw



100%|█████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 2270623.21it/s]


Extracting ./mnist_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST\raw

Train Epoch: 1 | Batch Holati: 0/60000 (0%) | Loss: 2.306896
Train Epoch: 1 | Batch Holati: 640/60000 (1%) | Loss: 2.299802
Train Epoch: 1 | Batch Holati: 1280/60000 (2%) | Loss: 2.306980
Train Epoch: 1 | Batch Holati: 1920/60000 (3%) | Loss: 2.292314
Train Epoch: 1 | Batch Holati: 2560/60000 (4%) | Loss: 2.302998
Train Epoch: 1 | Batch Holati: 3200/60000 (5%) | Loss: 2.301192
Train Epoch: 1 | Batch Holati: 3840/60000 (6%) | Loss: 2.304115
Train Epoch: 1 | Batch Holati: 4480/60000 (7%) | Loss: 2.300461
Train Epoch: 1 | Batch Holati: 5120/60000 (9%) | Loss: 2.299643
Train Epoch: 1 | Batch Holati: 5760/60000 (10%) | Loss: 2.295333
Train Epoch: 1 | Batch Holati: 6400/60000 (11%) | Loss: 2.295171
Train Epoch: 1 | Batch Holati: 7040/60000 (12%) | Loss: 2.295480
Train Epoch: 1 | Batch Holati: 7680/60000 (13%) | Loss: 2.294799
Train Epoch: 1 | Batch Holati: 8320/60000 (14%) | Loss: 2.297585
Train Epoch: 1 |